In [16]:
import pandas as pd
import numpy as np

In [17]:
#import excel spread sheet of list of violations
violations_list = pd.read_excel("Codes' List of Violations 15 aug 2017.xlsx")

In [18]:
#check out the columns to see how they were importated and how NULL values are treated
violations_list.head(1)

,VIOLATION_ID,ORG_ID,VIOLATION_TYPE,VIOLATION_DESC,VIOLATION_TEXT,REMEDIAL_TEXT,TABLE_NAME,EXPIRED_FLAG,DATE_EXPIRED,CREATED_BY,DATE_CREATED,MODIFIED_BY,DATE_MODIFIED,FEE_SETUP_ID,Unnamed: 14
0,1,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN


In [19]:
#check violation description for duplicates to start cutting down on list
violations_list.VIOLATION_DESC.value_counts(dropna = False)

N                                           137
NaN                                           9
LANDLORD REGISTRATION                         2
TRASH CANS                                    2
Failure to pay                                2
STAGNANT WATER                                2
Hazardous, pathogenic, radioactive waste      2
RAT HARBORAGE GARBAGE                         2
SIGN PERMIT REQUIRED                          2
Distance                                      2
Consignment Sale                              2
False statement on application                2
EXCESS VEGETATION                             2
CONSIGNMENT SALE                              2
JUNK TRASH RAT HARBORAGE                      2
SEWAGE ON GROUND                              2
FAULTY PLUMBING                               2
ELECTRICAL SERVICE PANEL                      2
Film, pics, reproductions depicting acts      1
Plumbing Systems – General Requirements       1
Homemade Beer Contest Fees              

In [20]:
#check how complete each column is. VIOLATION_DESC seems to be most complete with only 9 null objects (compared to range index)
violations_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 15 columns):
VIOLATION_ID      928 non-null object
ORG_ID            789 non-null object
VIOLATION_TYPE    788 non-null object
VIOLATION_DESC    925 non-null object
VIOLATION_TEXT    666 non-null object
REMEDIAL_TEXT     345 non-null object
TABLE_NAME        137 non-null datetime64[ns]
EXPIRED_FLAG      651 non-null object
DATE_EXPIRED      2 non-null object
CREATED_BY        650 non-null float64
DATE_CREATED      651 non-null object
MODIFIED_BY       23 non-null object
DATE_MODIFIED     22 non-null datetime64[ns]
FEE_SETUP_ID      0 non-null float64
Unnamed: 14       0 non-null float64
dtypes: datetime64[ns](2), float64(3), object(10)
memory usage: 109.5+ KB


In [21]:
#drop columns: ORG_ID, TABLE_NAME, EXPIRED_FLAG, DATE_EXPIRED, CREATED_BY, DATE_CREATED, MODIFIED_BY, DATE_MODIFIED, FEE_SETUP_ID, Unnamed:14
violations_trimmed = violations_list.drop(["ORG_ID", "TABLE_NAME", "EXPIRED_FLAG", "DATE_EXPIRED", "CREATED_BY", "DATE_CREATED", "MODIFIED_BY", "DATE_MODIFIED", "FEE_SETUP_ID", "Unnamed: 14"], axis=1)

In [22]:
#check to make sure columns were dropped as expected.
violations_trimmed.head(1)

,VIOLATION_ID,VIOLATION_TYPE,VIOLATION_DESC,VIOLATION_TEXT,REMEDIAL_TEXT
0,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS


In [23]:
#VIOLATION_TEXT column contains duplicates, need to clear that up first.
violations_trimmed["VIOLATION_TEXT"].value_counts()

See SW Code                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [24]:
violations_trimmed.columns = ["id", "type", "desc", "text", "remedial"]
violations_renamed = violations_trimmed

In [25]:
violations_renamed.head(1)

,id,type,desc,text,remedial
0,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS


In [26]:
violations_renamed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 5 columns):
id          928 non-null object
type        788 non-null object
desc        925 non-null object
text        666 non-null object
remedial    345 non-null object
dtypes: object(5)
memory usage: 36.6+ KB


In [27]:
#subset of rows containing "Section #." regular expression in text column
violations_section = violations_renamed[violations_renamed.text.str.match(r"(S|s)ection\s(\d+)\.", na=False)]
#remove 
violations_nosection = violations_renamed.drop(violations_section.index)
#subset of rows with variations of TCA in text column
violations_TCA = violations_nosection[violations_nosection.text.str.match("(T.C.A|Tennessee Code Annotated)", na=False)]
violations_noTCA = violations_nosection.drop(violations_TCA.index)
#subset of rows with variations of MCL in text column. Be aware MCL and MCL Chapter appear to have different types of codes
violations_MCL = violations_noTCA[violations_noTCA.text.str.match("(M.C.L|MCL Chapter)", na=False)]
violations_noMCL = violations_noTCA.drop(violations_MCL.index)
#subset of rows containing "-" in id column. These are errors.
violations_dash = violations_noMCL[violations_noMCL['id'].str.match("-", na=False)]
violations_dash = violations_noMCL.drop(violations_dash.index)
#subset of rows containing strings in id column. These values may be related to other entries or misplaced.
violations_description_id = violations_noMCL[violations_noMCL['id'].str.match("(\(|[a-zA-Z]|[0-9.])", na=False)]
violations_description_id = violations_dash.drop(violations_description_id.index)
#subset of rows containing NaN in type column
violations_NaN_type = violations_description_id[violations_description_id['type'].isnull()]
violations_NaN_type = violations_description_id.drop(violations_description_id[violations_description_id['type'].isnull()].index)


In [28]:
violations_section17.shape

(429, 5)

In [29]:
violations_nosection = violations_renamed.drop(violations_renamed[violations_renamed.text.str.match(r"(S|s)ection\s(\d+)\.", na=False)].index)

In [30]:
violations_nosection

,id,type,desc,text,remedial
15,(B). It is unlawful for any person to dispose ...,NaN,N,NaN,1
106,107,LANDREG,Landlord Registration Required,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,CAAR_LAND_REG
126,127,MVENVEND,Mobile Vending,Ordinance BL2006-1283 - Mobile Vendor: The ped...,CAAZ_MVEN_VENDING
196,197,TRASHCANS,Trash Cans,Ordinance 89-826 - Trash Cans: Refuse containe...,CAAH_TRASH_CANS
209,210,SWVIOLPTP,Violations - Post Construction Treatment,See SW Code,NaN
210,212,SWVIOLWQ,Violations for Water Quality Impacts,See SW Code,NaN
211,213,SWENVCRT,Environmental Court Proceedings,See SW Code,NaN
212,214,SWVIOLGP,Violations - for Construction Activities,See SW Code,NaN
213,247,917,HOUSE DISREPAIR,NaN,NaN
215,384,CAAA_BOAT,BOAT STORAGE IN FRONT YARD PROHIBITED,No trailer or watercraft shall be stored in th...,NaN


In [31]:
violations_TCA = violations_nosection[violations_nosection.text.str.match("(T.C.A|Tennessee Code Annotated)", na=False)]

In [32]:
violations_TCA

,id,type,desc,text,remedial
106,107,LANDREG,Landlord Registration Required,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,CAAR_LAND_REG
376,546,CAAR_LAND_REG,LANDLORD REGISTRATION REQUIRED,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,NaN
807,853,BT575101A2,Not Manufacturer and Wholesale Retail,T.C.A 57-5-101(a)(2): Except as otherwise prov...,NaN
808,854,BT575101A3,Not Wholesale and Manufacturer Retail,T.C.A 57-5-101(a)(3): Except as otherwise prov...,NaN
809,855,BT575103A1,Operating without county city permit,T.C.A 57-5-103(a)(1): It is unlawful to operat...,NaN
810,856,BT575103A10,Permits to citizen legal resident,"T.C.A 57-5-103(a)(10): After July 1, 2015, a c...",NaN
811,857,BT575103A2,Permit to business owner,T.C.A 57-5-103(a)(2): Permits shall be issued ...,NaN
812,858,BT575103A3A,Cannot transfer ownership,T.C.A 57-5-103(a)(3)(A): A permit shall be val...,NaN
813,859,BT575103A3B,Permit for single location only,T.C.A 57-5-103(a)(3)(B): A permit shall be val...,NaN
814,860,BT575103A3C,Business Name same as Permit Name,T.C.A 57-5-103(a)(3)(C): A permit shall be val...,NaN


In [33]:
violations_noTCA = violations_nosection.drop(violations_nosection[violations_nosection.text.str.match(r"(T.C.A|Tennessee Code Annotated)", na=False)].index)

In [34]:
violations_noTCA

,id,type,desc,text,remedial
15,(B). It is unlawful for any person to dispose ...,NaN,N,NaN,1
126,127,MVENVEND,Mobile Vending,Ordinance BL2006-1283 - Mobile Vendor: The ped...,CAAZ_MVEN_VENDING
196,197,TRASHCANS,Trash Cans,Ordinance 89-826 - Trash Cans: Refuse containe...,CAAH_TRASH_CANS
209,210,SWVIOLPTP,Violations - Post Construction Treatment,See SW Code,NaN
210,212,SWVIOLWQ,Violations for Water Quality Impacts,See SW Code,NaN
211,213,SWENVCRT,Environmental Court Proceedings,See SW Code,NaN
212,214,SWVIOLGP,Violations - for Construction Activities,See SW Code,NaN
213,247,917,HOUSE DISREPAIR,NaN,NaN
215,384,CAAA_BOAT,BOAT STORAGE IN FRONT YARD PROHIBITED,No trailer or watercraft shall be stored in th...,NaN
226,395,CAAA_LANDLORD_REGSTR,LANDLORD REGISTRATION,Failure to register as a landlord. T.C.A. 66-...,NaN


In [35]:
violations_MCL = violations_noTCA[violations_noTCA.text.str.match("(M.C.L|MCL Chapter)", na=False)]

In [36]:
violations_MCL

,id,type,desc,text,remedial
481,627,CAGA_DEX,DOG EXCREMENT,MCL Chapter 8.04.180 – Removal of Excrement. ...,NaN
482,628,CAGB_HAZ_W,HAZARDOUS WASTE,"MCL Chapter 10.20.150 - Hazardous, pathogenic ...",NaN
483,629,CAGC_GCL,GARBAGE CONTAINER LIDS,MCL Chapter 10.20.160 – Container Requirements...,NaN
484,630,CAGD_ID,ILLEGAL DUMPING,MCL Chapter 10.20.320 – Illegal Dumping. It i...,NaN
485,631,CAGE_LPR,LITTER ON PRIVATE PROPERTY,MCL Chapter 10.24.070 - Litter on private prop...,NaN
486,632,CAGF_LPU,LITTER ON PUBLIC PROPERTY,MCL Chapter 10.24.140 - Litter on vacant lots....,NaN
487,633,CAGG_LVH,LITTER FROM VEHICLES,MCL Chapter 10.24.150 - Litter from vehicles--...,NaN
488,634,CAGH_HWG,EXCESS VEGETATION,MCL Chapter 10.26.010 – Every premise includin...,NaN
489,635,CAGI_GCAN,GARBAGE APPROVED CONTAINERS,MCL Chapter 10.32.120 - All garbage and trash ...,NaN
490,636,CAGJ_GT_RATS,RAT HARBORAGE GARBAGE,MCL Chapter 10.32.130 – Permitting garbage and...,NaN


In [37]:
violations_noMCL = violations_noTCA.drop(violations_noTCA[violations_noTCA.text.str.match("(M.C.L|MCL Chapter)", na=False)].index)

In [38]:
violations_noMCL

,id,type,desc,text,remedial
15,(B). It is unlawful for any person to dispose ...,NaN,N,NaN,1
126,127,MVENVEND,Mobile Vending,Ordinance BL2006-1283 - Mobile Vendor: The ped...,CAAZ_MVEN_VENDING
196,197,TRASHCANS,Trash Cans,Ordinance 89-826 - Trash Cans: Refuse containe...,CAAH_TRASH_CANS
209,210,SWVIOLPTP,Violations - Post Construction Treatment,See SW Code,NaN
210,212,SWVIOLWQ,Violations for Water Quality Impacts,See SW Code,NaN
211,213,SWENVCRT,Environmental Court Proceedings,See SW Code,NaN
212,214,SWVIOLGP,Violations - for Construction Activities,See SW Code,NaN
213,247,917,HOUSE DISREPAIR,NaN,NaN
215,384,CAAA_BOAT,BOAT STORAGE IN FRONT YARD PROHIBITED,No trailer or watercraft shall be stored in th...,NaN
226,395,CAAA_LANDLORD_REGSTR,LANDLORD REGISTRATION,Failure to register as a landlord. T.C.A. 66-...,NaN


In [39]:
violations_dash = violations_noMCL[violations_noMCL['id'].str.match("-", na=False)]
violations_dash

,id,type,desc,text,remedial
441,-,NaN,N,NaN,2
443,-,NaN,N,NaN,2
445,-,NaN,N,NaN,2
447,-,NaN,N,NaN,2
449,-,NaN,N,NaN,2
451,-,NaN,N,NaN,2
453,-,NaN,N,NaN,2
455,-,NaN,N,NaN,2
457,-,NaN,N,NaN,2
459,-,NaN,N,NaN,2


In [40]:
violations_dash = violations_noMCL.drop(violations_dash.index)

In [41]:
violations_dash

,id,type,desc,text,remedial
15,(B). It is unlawful for any person to dispose ...,NaN,N,NaN,1
126,127,MVENVEND,Mobile Vending,Ordinance BL2006-1283 - Mobile Vendor: The ped...,CAAZ_MVEN_VENDING
196,197,TRASHCANS,Trash Cans,Ordinance 89-826 - Trash Cans: Refuse containe...,CAAH_TRASH_CANS
209,210,SWVIOLPTP,Violations - Post Construction Treatment,See SW Code,NaN
210,212,SWVIOLWQ,Violations for Water Quality Impacts,See SW Code,NaN
211,213,SWENVCRT,Environmental Court Proceedings,See SW Code,NaN
212,214,SWVIOLGP,Violations - for Construction Activities,See SW Code,NaN
213,247,917,HOUSE DISREPAIR,NaN,NaN
215,384,CAAA_BOAT,BOAT STORAGE IN FRONT YARD PROHIBITED,No trailer or watercraft shall be stored in th...,NaN
226,395,CAAA_LANDLORD_REGSTR,LANDLORD REGISTRATION,Failure to register as a landlord. T.C.A. 66-...,NaN


In [42]:
violations_description_id = violations_noMCL[violations_noMCL['id'].str.match("(\(|[a-zA-Z]|[0-9.])", na=False)]
violations_description_id

,id,type,desc,text,remedial
15,(B). It is unlawful for any person to dispose ...,NaN,N,NaN,1
438,he or she shall serve notice of the risk and t...,NaN,NaN,NaN,NaN
439,tree at the expense of the property owner,NaN,N,NaN,2
707,"human occupation or use, upon failure or refus...",NaN,NaN,NaN,NaN
708,"remove or demolish, such dwelling or structure...",NaN,N,NaN,2
895,4. A valid identification card showing a re...,NaN,N,NaN,10635
897,A. Department of Health,NaN,NaN,NaN,NaN
898,B. Fire Marshal Approval,NaN,NaN,NaN,NaN
899,C. Certificate of Registration or Tax Exempt...,NaN,NaN,NaN,NaN
900,D. Letter from charitable organization (only ...,NaN,NaN,NaN,NaN


In [43]:
violations_description_id = violations_dash.drop(violations_description_id.index)

In [44]:
violations_description_id

,id,type,desc,text,remedial
126,127,MVENVEND,Mobile Vending,Ordinance BL2006-1283 - Mobile Vendor: The ped...,CAAZ_MVEN_VENDING
196,197,TRASHCANS,Trash Cans,Ordinance 89-826 - Trash Cans: Refuse containe...,CAAH_TRASH_CANS
209,210,SWVIOLPTP,Violations - Post Construction Treatment,See SW Code,NaN
210,212,SWVIOLWQ,Violations for Water Quality Impacts,See SW Code,NaN
211,213,SWENVCRT,Environmental Court Proceedings,See SW Code,NaN
212,214,SWVIOLGP,Violations - for Construction Activities,See SW Code,NaN
213,247,917,HOUSE DISREPAIR,NaN,NaN
215,384,CAAA_BOAT,BOAT STORAGE IN FRONT YARD PROHIBITED,No trailer or watercraft shall be stored in th...,NaN
226,395,CAAA_LANDLORD_REGSTR,LANDLORD REGISTRATION,Failure to register as a landlord. T.C.A. 66-...,NaN
236,405,CAAA_TRASH_CANS,TRASH CANS,Ord. 89-826 - Trash Cans: Refuse containers sh...,NaN


In [45]:
violations_NaN_type = violations_description_id[violations_description_id['type'].isnull()]
violations_NaN_type

,id,type,desc,text,remedial
399,NaN,NaN,N,NaN,2
401,NaN,NaN,N,NaN,2
411,NaN,NaN,N,NaN,2
421,NaN,NaN,N,NaN,2
433,NaN,NaN,N,NaN,2
524,NaN,NaN,N,NaN,2


In [46]:
violations_NaN_type = violations_description_id.drop(violations_description_id[violations_description_id['type'].isnull()].index)
with pd.option_context('display.max_rows', None, 'display.max_columns', 10):
    print(violations_NaN_type)

      id                  type                                      desc  \
126  127              MVENVEND                            Mobile Vending   
196  197             TRASHCANS                                Trash Cans   
209  210             SWVIOLPTP  Violations - Post Construction Treatment   
210  212              SWVIOLWQ      Violations for Water Quality Impacts   
211  213              SWENVCRT           Environmental Court Proceedings   
212  214              SWVIOLGP  Violations - for Construction Activities   
213  247                   917                           HOUSE DISREPAIR   
215  384             CAAA_BOAT     BOAT STORAGE IN FRONT YARD PROHIBITED   
226  395  CAAA_LANDLORD_REGSTR                     LANDLORD REGISTRATION   
236  405       CAAA_TRASH_CANS                                TRASH CANS   
350  520       CAAH_TRASH_CANS                                TRASH CANS   
380  550      CAAZ_CERT_COMPLY                CERTIFICATE COMPLIANCE REQ   
394  564    